In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from copy import deepcopy
df_conf = pd.read_csv("sinan_conf.csv", index_col="Unnamed: 0")
df_conf

c:\users\lucas\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (3,10,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID_AGRAVO,DT_NOTIFIC,DT_NASC,NU_ANO,DT_SIN_PRI,NU_IDADE_N,CS_SEXO,CS_ESCOL_N,ID_MN_RESI,NDUPLIC_N,DT_INIC_TR,CLA_TIPO_N,TRATAMENTO,SITUA_ENCE,NU_NOTIFIC
1,B659,2013-01-02,1947-09-30,2013,2012-12-27,4065.0,M,NaN,316400,NaN,NaN,NaN,NaN,NaN,NaN
2,B659,2013-01-02,1982-11-10,2013,2012-11-23,4030.0,M,6.0,311760,NaN,NaN,NaN,NaN,NaN,NaN
3,B659,2013-01-02,1962-08-23,2013,2012-12-20,4050.0,F,NaN,311940,NaN,NaN,NaN,NaN,NaN,NaN
4,B659,2013-01-02,1979-08-03,2013,2012-12-25,4033.0,M,NaN,313090,NaN,NaN,NaN,NaN,NaN,NaN
5,B659,2013-01-03,NaN,2013,2012-12-04,4009.0,M,1.0,310620,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58830,A169,2021-01-01,NaN,2021,2020-12-31,4046.0,M,9.0,311860,NaN,18628.0,NaN,1,NaN,NaN
58831,A169,2021-01-05,NaN,2021,2020-12-31,4041.0,M,2.0,313290,NaN,18633.0,NaN,1,NaN,NaN
58832,A169,2021-01-07,NaN,2021,2020-12-31,4058.0,F,9.0,311860,NaN,18634.0,NaN,1,NaN,NaN
58833,A16.,2020-09-16,NaN,2020,2020-09-16,NaN,M,1.0,314430,.,NaN,NaN,.,.,NaN


In [2]:
query = "   (ID_AGRAVO == 'B659')" + \
        "or (ID_AGRAVO == 'A169' and TRATAMENTO == '1' and SITUA_ENCE != '6')" + \
        "or (ID_AGRAVO == 'B551' and CLA_TIPO_N == '1')"

df_conf = df_conf.query(query)
df_conf

,ID_AGRAVO,DT_NOTIFIC,DT_NASC,NU_ANO,DT_SIN_PRI,NU_IDADE_N,CS_SEXO,CS_ESCOL_N,ID_MN_RESI,NDUPLIC_N,DT_INIC_TR,CLA_TIPO_N,TRATAMENTO,SITUA_ENCE,NU_NOTIFIC
1,B659,2013-01-02,1947-09-30,2013,2012-12-27,4065.0,M,NaN,316400,NaN,NaN,NaN,NaN,NaN,NaN
2,B659,2013-01-02,1982-11-10,2013,2012-11-23,4030.0,M,6.0,311760,NaN,NaN,NaN,NaN,NaN,NaN
3,B659,2013-01-02,1962-08-23,2013,2012-12-20,4050.0,F,NaN,311940,NaN,NaN,NaN,NaN,NaN,NaN
4,B659,2013-01-02,1979-08-03,2013,2012-12-25,4033.0,M,NaN,313090,NaN,NaN,NaN,NaN,NaN,NaN
5,B659,2013-01-03,NaN,2013,2012-12-04,4009.0,M,1.0,310620,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58828,A169,2020-12-31,NaN,2020,2020-12-31,4038.0,M,NaN,313250,NaN,18627.0,NaN,1,NaN,NaN
58829,A169,2021-01-01,NaN,2021,2020-12-31,4049.0,M,NaN,310620,NaN,18627.0,NaN,1,NaN,NaN
58830,A169,2021-01-01,NaN,2021,2020-12-31,4046.0,M,9.0,311860,NaN,18628.0,NaN,1,NaN,NaN
58831,A169,2021-01-05,NaN,2021,2020-12-31,4041.0,M,2.0,313290,NaN,18633.0,NaN,1,NaN,NaN


In [3]:
def count_duplicates(df, agravos, dupli_cols):
    for agravo in agravos:
        aux_df = df.query("ID_AGRAVO == @agravo")
        print(agravo)
        print(Counter(aux_df.groupby(dupli_cols).size()))
        
def process_duplicates(df, cols_por_agravo):
    aux_df = pd.DataFrame(columns = df.columns)

    df_list = []
    for agravo, dupli_cols in cols_por_agravo.items():
        count_duplicates(df, agravo, dupli_cols)
    
        if len(agravo) > 1:
            aux = df[df['ID_AGRAVO'].isin(agravo)].drop_duplicates(dupli_cols)
            df_list.append(aux.copy(deep=True))
        else:
            aux = df[df['ID_AGRAVO'] == agravo[0]].drop_duplicates(dupli_cols)
            df_list.append(aux.copy(deep=True))

    return aux_df.append(df_list, ignore_index=True)

In [4]:
cols_por_agravo = {
    ('B551',): ['NU_NOTIFIC', 'ID_MN_RESI', 'DT_NASC', 'CS_SEXO', 'DT_SIN_PRI', 'DT_NOTIFIC', 'DT_INIC_TR'],
    ('B659',): ['ID_MN_RESI', 'DT_NASC', 'CS_SEXO', 'DT_SIN_PRI'],
    ('A169',): ['ID_MN_RESI', 'DT_NASC', 'CS_SEXO', 'DT_SIN_PRI', 'DT_INIC_TR', 'DT_NOTIFIC'],
}

In [5]:
df_conf.sort_values(by=['DT_NOTIFIC'], inplace=True)
df_conf = df_conf[df_conf['NDUPLIC_N'] != "2"]

aux_conf = process_duplicates(df_conf, cols_por_agravo)

aux_conf.drop(['TRATAMENTO', 'SITUA_ENCE', 'CLA_TIPO_N', 'NU_NOTIFIC', 'NDUPLIC_N', 'DT_INIC_TR'], axis=1, inplace=True)
aux_conf

B551
Counter()
B659
Counter({1: 15337, 2: 53})
A169
Counter()


<ipython-input-5-2472f169d1ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conf.sort_values(by=['DT_NOTIFIC'], inplace=True)


,ID_AGRAVO,DT_NOTIFIC,DT_NASC,NU_ANO,DT_SIN_PRI,NU_IDADE_N,CS_SEXO,CS_ESCOL_N,ID_MN_RESI
0,B551,2013-01-02,NaN,2013,2013-01-02,4024.0,F,6.0,310620
1,B551,2013-01-02,NaN,2013,2012-11-02,4014.0,M,2.0,317000
2,B551,2013-01-03,NaN,2013,2012-10-10,4036.0,F,NaN,310660
3,B551,2013-01-03,NaN,2013,2012-12-28,4048.0,M,9.0,316860
4,B551,2013-01-03,NaN,2013,2012-12-12,4045.0,F,6.0,314330
...,...,...,...,...,...,...,...,...,...
43800,A169,2021-01-11,NaN,2021,2020-01-08,4053.0,M,3.0,314840
43801,A169,2021-01-14,NaN,2021,2020-12-28,4069.0,M,NaN,310620
43802,A169,2021-01-15,NaN,2021,2020-12-23,4051.0,M,2.0,316960
43803,A169,2021-01-18,NaN,2021,2020-12-30,4024.0,M,5.0,310710


In [6]:
df_sus = pd.read_csv("sinan_sus.csv")

df_sus

c:\users\lucas\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,ID_AGRAVO,DT_NOTIFIC,DT_NASC,NU_ANO,DT_SIN_PRI,NU_IDADE_N,CS_SEXO,CS_ESCOL_N,ID_MN_RESI,NDUPLIC_N,DT_INVEST,CLASSI_FIN,ENTRADA,TP_UNI_EXT,TRATAMENTO
0,1,B571,2013-01-09,1954-03-01,2013.0,2013-01-09,4058.0,M,1.0,314710,NaN,2013-01-09,2.0,NaN,NaN,NaN
1,2,B571,2013-04-22,1986-01-22,2013.0,2013-04-22,4027.0,M,6.0,312560,NaN,2013-04-22,2.0,NaN,NaN,NaN
2,3,B571,2013-05-16,1977-10-04,2013.0,2013-05-13,4035.0,M,3.0,314710,NaN,2013-05-16,2.0,NaN,NaN,NaN
3,4,B571,2013-01-08,NaN,2013.0,2013-01-08,4066.0,M,1.0,315600,NaN,2013-01-08,8.0,NaN,NaN,NaN
4,5,B571,2013-11-01,1989-05-15,2013.0,2013-11-01,4024.0,F,8.0,316770,NaN,2013-11-01,8.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2698994,2698995,Y09,2019-03-26,1990-05-31,2019.0,2019-03-26,4028.0,F,8.0,317130,NaN,NaN,NaN,NaN,NaN,NaN
2698995,2698996,Y09,2019-03-26,1983-10-22,2019.0,2019-03-18,4035.0,F,NaN,314000,NaN,NaN,NaN,NaN,NaN,NaN
2698996,2698997,Y09,2019-03-26,1991-07-20,2019.0,2019-03-23,4027.0,F,3.0,314000,NaN,NaN,NaN,NaN,NaN,NaN
2698997,2698998,Y09,2019-03-26,1998-12-30,2019.0,2019-03-24,4020.0,F,2.0,313190,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
query = "   (ID_AGRAVO not in ['A959', 'A90', 'A91', 'A92', 'B550'] and CLASSI_FIN == 1)" + \
        "or (ID_AGRAVO == 'A959' and CLASSI_FIN in [1, 2])" + \
        "or (ID_AGRAVO in ['A90', 'A91', 'A92'] and CLASSI_FIN in [10, 11, 12, 13]) " + \
        "or (ID_AGRAVO == 'B550' and CLASSI_FIN == 1 and ENTRADA == 1)"

df_sus = df_sus.query(query)

df_sus.drop(['CLASSI_FIN', 'DT_INVEST', 'Unnamed: 0'], axis=1, inplace=True)

df_sus

c:\users\lucas\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,ID_AGRAVO,DT_NOTIFIC,DT_NASC,NU_ANO,DT_SIN_PRI,NU_IDADE_N,CS_SEXO,CS_ESCOL_N,ID_MN_RESI,NDUPLIC_N,ENTRADA,TP_UNI_EXT,TRATAMENTO
1433,A90,2013-04-05,1939-04-04,2013.0,2013-04-03,4073.0,M,1.0,310090,NaN,NaN,NaN,NaN
2579,A90,2013-01-07,1954-06-09,2013.0,2012-12-31,4058.0,F,1.0,316930,NaN,NaN,NaN,NaN
3058,A90,2013-01-08,1971-11-05,2013.0,2013-01-06,4041.0,F,6.0,310090,NaN,NaN,NaN,NaN
4479,A90,2013-01-09,1990-05-15,2013.0,2013-01-06,4022.0,M,NaN,311300,NaN,NaN,NaN,NaN
11311,A90,2013-03-04,1965-09-05,2013.0,2013-03-04,4047.0,F,0.0,311300,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2636525,Y09,2018-06-17,2003-11-26,2018.0,2018-06-17,4014.0,F,3.0,310620,NaN,NaN,NaN,NaN
2640979,Y09,2018-02-22,2000-03-24,2018.0,2018-02-11,4017.0,F,5.0,310620,NaN,NaN,NaN,NaN
2643289,Y09,2018-01-02,2014-01-28,2018.0,2017-12-31,4003.0,M,10.0,315780,NaN,NaN,NaN,NaN
2643310,Y09,2018-07-10,1950-12-14,2018.0,2018-07-10,4067.0,M,3.0,310620,NaN,NaN,NaN,NaN


In [8]:
cols_por_agravo = {
    ('A90', 'A91', 'A92'): ['ID_MN_RESI', 'DT_NASC', 'CS_SEXO', 'DT_SIN_PRI', 'DT_NOTIFIC'],
    ('A279',): ['ID_MN_RESI', 'DT_NASC', 'CS_SEXO', 'DT_SIN_PRI'],
    ('T659',): ['ID_MN_RESI', 'DT_NASC', 'CS_SEXO', 'DT_SIN_PRI'],
    ('B54',): ['ID_MN_RESI', 'DT_NASC', 'CS_SEXO', 'DT_SIN_PRI', 'DT_NOTIFIC'],
    ('Y09',): ['ID_MN_RESI', 'DT_NASC', 'CS_SEXO', 'DT_SIN_PRI', 'DT_NOTIFIC'],
    ('B550',): ['ID_MN_RESI', 'DT_NASC', 'CS_SEXO', 'DT_SIN_PRI', 'DT_NOTIFIC', 'TRATAMENTO'],
    ('A010',): ['ID_MN_RESI', 'DT_NASC', 'CS_SEXO', 'DT_SIN_PRI', 'DT_NOTIFIC']
}

In [9]:
df_sus.sort_values(by=['DT_NOTIFIC'], inplace=True)
df_sus = df_sus[df_sus['NDUPLIC_N'] != "2"]

aux_sus = process_duplicates(df_sus, cols_por_agravo)
aux_sus.drop(['ENTRADA', 'TP_UNI_EXT', 'TRATAMENTO', 'NDUPLIC_N'], axis=1, inplace=True)
aux_sus

<ipython-input-9-584c170d09d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sus.sort_values(by=['DT_NOTIFIC'], inplace=True)


A90
Counter({1: 1022087, 2: 4317, 3: 61, 4: 4})
A91
Counter()
A92
Counter()
A279
Counter({1: 1026, 2: 6})
T659
Counter({1: 80409, 2: 518, 3: 6})
B54
Counter({1: 428, 2: 2, 4: 1})
Y09
Counter({1: 44556, 2: 123, 3: 1})
B550
Counter({1: 2660})
A010
Counter({1: 6})


,ID_AGRAVO,DT_NOTIFIC,DT_NASC,NU_ANO,DT_SIN_PRI,NU_IDADE_N,CS_SEXO,CS_ESCOL_N,ID_MN_RESI
0,A90,2013-01-03,1974-11-08,2013.0,2013-01-02,4038.0,F,9.0,310090
1,A90,2013-01-03,1997-07-19,2013.0,2013-01-03,4015.0,F,5.0,310090
2,A90,2013-01-05,1952-07-27,2013.0,2013-01-05,4060.0,F,9.0,314790
3,A90,2013-01-07,1954-06-09,2013.0,2012-12-31,4058.0,F,1.0,316930
4,A90,2013-01-07,1949-02-15,2013.0,2013-01-07,4063.0,F,9.0,310090
...,...,...,...,...,...,...,...,...,...
1169239,A010,2017-11-06,1990-02-05,2017.0,2017-10-30,4027.0,F,9.0,310620
1169240,A010,2018-02-20,NaN,2018.0,2018-02-17,4080.0,M,9.0,313760
1169241,A010,2018-04-03,NaN,2018.0,2018-03-29,4061.0,M,9.0,311230
1169242,A010,2018-04-04,NaN,2018.0,2018-03-30,4050.0,F,9.0,315570


In [10]:
df = aux_sus.append(aux_conf, ignore_index=True)

df

,ID_AGRAVO,DT_NOTIFIC,DT_NASC,NU_ANO,DT_SIN_PRI,NU_IDADE_N,CS_SEXO,CS_ESCOL_N,ID_MN_RESI
0,A90,2013-01-03,1974-11-08,2013,2013-01-02,4038.0,F,9.0,310090
1,A90,2013-01-03,1997-07-19,2013,2013-01-03,4015.0,F,5.0,310090
2,A90,2013-01-05,1952-07-27,2013,2013-01-05,4060.0,F,9.0,314790
3,A90,2013-01-07,1954-06-09,2013,2012-12-31,4058.0,F,1.0,316930
4,A90,2013-01-07,1949-02-15,2013,2013-01-07,4063.0,F,9.0,310090
...,...,...,...,...,...,...,...,...,...
1213044,A169,2021-01-11,NaN,2021,2020-01-08,4053.0,M,3.0,314840
1213045,A169,2021-01-14,NaN,2021,2020-12-28,4069.0,M,NaN,310620
1213046,A169,2021-01-15,NaN,2021,2020-12-23,4051.0,M,2.0,316960
1213047,A169,2021-01-18,NaN,2021,2020-12-30,4024.0,M,5.0,310710


In [ ]:
df.to_csv()